In [1]:
import numpy as np
from tqdm import tqdm
import requests as rq
from astropy.io import fits

In [10]:
fits_image_filename = '/Users/juliusgassert/full_binned/bV230621-ligodesi_S230619bg_070355_624736_r_wfi_230621_010.fits'
hdul = fits.open(fits_image_filename)  # open a FITS file
hdr = hdul[0].header  # the primary HDU header
print(hdr)

SIMPLE  =                    T                                                  BITPIX  =                  -32                                                  NAXIS   =                    2                                                  NAXIS1  =                  830                                                  NAXIS2  =                  871                                                  EXTEND  =                    T                                                  CRVAL1  =       105.8919283288 / World                                          CRVAL2  =       62.82787571848 / World                                          CTYPE1  = 'RA---TAN'           / CTYPE1                                         CTYPE2  = 'DEC--TAN'           / CTYPE2                                         CRPIX1  =        415.123692576                                                  CRPIX2  =       435.8970021587                                                  CD1_1   =  -0.0005535604280931          

In [15]:
hdr[6],hdr[7]

(105.8919283288, 62.82787571848)

In [17]:
test_name = "hello" + str(hdr[6])

In [18]:
test_name

'hello105.8919283288'

In [53]:
start_num = 10
end_num = 11

FOV = 30 # 30 for WWFI, 7 for 3kk

# calculate needed download size
curr_pixscale = 0.2 #ideal pixscale from LS
size = FOV*60/curr_pixscale
if size > 10000:
    print(f"original pixscale of {curr_pixscale} produces a too large image with size {size}")
    size = 3000
    curr_pixscale = FOV*60/size
print(f"using size of {size} with pixscale of {curr_pixscale} to get the same FOV")


event_id = "ligodesi_S230619bg"
general_file_name = f"bV230621-{event_id}_070355_624736_r_wfi_230621_0"
file_path_prefix = f"/Users/juliusgassert/full_binned/{general_file_name}"
file_ext = ".fits"

save_path_prefix = "/Users/juliusgassert/full_binned/LS/"

bands = "griz"
pixscale_download = curr_pixscale
download_filetype = "fits"

download_url_prefix = f"https://www.legacysurvey.org/viewer/" + download_filetype + "-cutout?"

#ra=190.1086&dec=1.2005&layer=ls-dr10&pixscale=0.262&bands=griz"

using size of 9000.0 with pixscale of 0.2 to get the same FOV


In [54]:
file_name_num = np.arange(start_num, end_num, 1)

for num in tqdm(file_name_num):
    file_path = file_path_prefix + str(num) + file_ext
    hdul = fits.open(file_path)
    hdr = hdul[0].header
    RA, dec = hdr[6], hdr[7]

    download_url = download_url_prefix + "ra=" + str(RA) + "&dec=" + str(dec) + "&layer=ls-dr10&pixscale=" + str(pixscale_download) + "&bands=" + bands + "&size=" + str(int(size))
    print(download_url)
    response = rq.get(download_url, allow_redirects = True)
    
    if response.status_code == 200:
        save_path = save_path_prefix + event_id + "_bands=" + bands + "." + download_filetype
        with open(save_path, 'wb') as f:
            f.write(response.content)
    else:
        print("download failed with HTTP status: ", response.status_code)


  0%|          | 0/1 [00:00<?, ?it/s]

https://www.legacysurvey.org/viewer/fits-cutout?ra=105.8919283288&dec=62.82787571848&layer=ls-dr10&pixscale=0.2&bands=griz&size=9000


100%|██████████| 1/1 [00:17<00:00, 17.59s/it]
